In [50]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np

# to calculate KL-divergence
from scipy.stats import entropy

import math
from torchvision.models import inception_v3


net = inception_v3(pretrained=True).cuda()


In [51]:
# add the loadDataset function [required as an interface to custom dataloaders]
import sys
sys.path.append('../')
from Datasets.loadDataset import loadDataset, getChannels
reload(sys.modules[loadDataset.__module__])

cuda = 1

batchSize = 256

primaryDataset = 'SVHN'
numPrimaryOutputChannels = getChannels(primaryDataset)

classes = [0]
instances = [7000 for i in range(10)]
kwargs = {'num_workers': 1, 'pin_memory': True, 'drop_last':True} if cuda else {}


trainDatasetPrimary = loadDataset(primaryDataset, classes, instances,'train')

primaryTrainLoader = torch.utils.data.DataLoader(trainDatasetPrimary, batch_size=batchSize, shuffle=True, **kwargs)

In [52]:
class IgnoreLabelDataset(torch.utils.data.Dataset):
    def __init__(self, orig):
        self.orig = orig

    def __getitem__(self, index):
        return self.orig[index][0]

    def __len__(self):
        return len(self.orig)

In [54]:
def get_pred(x, model, resize=True):
    
    dtype = get_dtype()
    
    if resize:
        x = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)(x)
    x = model(x)
    return F.softmax(x).data.cpu().numpy()

In [55]:
def get_dtype():
    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor
    return dtype

In [56]:
 def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
    
    """
    Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """

    
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    dtype = get_dtype()
        
    # model to be used
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    
    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)


    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv, inception_model)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))
    
    # return the mean and standard deviation of inception score
    return np.mean(split_scores), np.std(split_scores)

if __name__ == '__main__':

    print ("Calculating Inception Score...")
    print (inception_score(IgnoreLabelDataset(trainDatasetPrimary), cuda=True, batch_size=32, resize=True, splits=10))

Calculating Inception Score...
(2.5737165545806788, 0.067893171940134556)
